In [40]:
# organize the imports
import opentrons.execute
from opentrons import types 
import time
import numpy as np


In [41]:
import opentrons
opentrons.__file__

'/usr/lib/python3.7/site-packages/opentrons/__init__.pyc'

In [42]:
#!pip3 install zeroconf
#!pip3 install requests

# Connect to the microscope 
from microscope_client import MicroscopeClient 
microscope = MicroscopeClient(host="192.168.178.33", port=5000)


In [43]:
# Connect to the opentrons robot
protocol = opentrons.execute.get_protocol_api('2.9')
protocol.home()

# setup the pipets
pipette = protocol.load_instrument('p300_single_gen2', 'right')
pipette_8 = protocol.load_instrument('p300_multi', 'left') # causes errors -> don't use two pipettes!!


In [44]:
# setup the labware in the trays
#plate_microscope = protocol.load_labware('corning_96_wellplate_360ul_flat', location='5')
plate_reagents = protocol.load_labware('corning_24_wellplate_3.4ml_flat', location='7')

# Move light to the sample 
tiprack = protocol.load_labware('opentrons_96_filtertiprack_200ul',location='8')

In [45]:
plate_reagents.well(1).center().__dict__

{'_point': Point(x=17.48, y=233.37, z=11.57),
 '_labware': B1 of Corning 24 Well Plate 3.4 mL Flat on 7}

In [53]:
from opentrons.types import Location, Point

# ######
# 0 4 8
# 1 5 9
# 2 6 10 
# 3 7 11


i_primary = 0
i_secondary = 1
i_bsa = 2
i_pbs = 3
i_trash = 4
i_mea = 5
V_aspirate = 200

offset_z_reagents = -20
# define locations 
minimum_z_height = 150 # minimum height for arcs driven by the robot
position_sample_light = (124+128, 163, 200)
position_sample =  (124+128,133,130)#
position_idle =  (200,200,140)#
position_primary = list(plate_reagents.well(i_primary).center()._point)
position_secondary = list(plate_reagents.well(i_secondary).center()._point)
position_bsa = list(plate_reagents.well(i_bsa).center()._point)
position_pbs = list(plate_reagents.well(i_pbs).center()._point)
position_mea = list(plate_reagents.well(i_mea).center()._point)
position_trash = list(plate_reagents.well(i_trash).center()._point)


def move_to_coord(pipette, position_xyz, offset = (0,0,0), minimum_z_height=190):
    if pipette._last_tip_picked_up_from is None:
        print("Please add a pipette first!")
        
        
    position_final = (position_xyz[0]+offset[0],
                        position_xyz[1]+offset[1],
                        position_xyz[2]+offset[2])
    print("Moving to: "+str(position_final))                       
    pipette.move_to(Location(Point(*position_final), None),minimum_z_height=minimum_z_height)


# set speed
protocol.max_speeds['X'] = 200 # default = 400
protocol.max_speeds['Y'] = 200 # default = 300

def pickup_fresh_pipette_tip(i_pipette, pipette, tiprack, z_offset=-3):
    try:
        my_fresh_tip = tiprack.wells()[i_pipette]
        adjusted_location = my_fresh_tip.center().move(types.Point(x=0, y=0, z=z_offset))
        pipette.pick_up_tip(adjusted_location)
    except Exception as e:
        print(e)
        i_pipette += 1
        print("no pipette attached")
        pipette.drop_tip()
        pipette.pick_up_tip(my_fresh_tip)

    return i_pipette


def takesnapshot(pipette, position_sample_light, offset=(2,-2,0)):
    microscope.set_laser_led(0,1)
    move_to_coord(pipette, position_sample_light, offset=offset, minimum_z_height=minimum_z_height)
    microscope.capture_image_to_disk()
    microscope.set_laser_led(0,0)

In [51]:
#protocol.home()
i_pipette = 0 
t_wait_primary = 120 # minutes
takesnapshot(pipette_8, position_sample_light, offset=(0,0,0))


Please add a pipette first!
Moving to: (252, 163, 200)


In [55]:
position_idle =  (100,200,140)#
move_to_coord(pipette, position_idle, offset=(0,0,offset_z_reagents), minimum_z_height=minimum_z_height)

Please add a pipette first!
Moving to: (100, 200, 120)


In [54]:
move_to_coord(pipette_8, position_sample_light, offset=(0,0,0), minimum_z_height=minimum_z_height)
microscope.set_laser_led(100,1)

Please add a pipette first!
Moving to: (252, 163, 200)


# Blocking

In [34]:
# pkcup primary
#pipette.home()
i_pipette = pickup_fresh_pipette_tip(i_pipette, pipette, tiprack, -8)

#i_pipette = pickup_fresh_pipette_tip(i_pipette, pipette, tiprack)
V_aspirate = 100

move_to_coord(pipette, position_bsa, offset=(0,0,offset_z_reagents), minimum_z_height=minimum_z_height)
pipette.aspirate(V_aspirate)

# add primary to sample
move_to_coord(pipette, position_sample, offset=(0,0,1), minimum_z_height=minimum_z_height)
pipette.dispense()
pipette.mix(3)

# shake a bit
x_shaking = 1
for _ in range(0):
    move_to_coord(pipette, position_sample, offset=(x_shaking,0,0), minimum_z_height=None)
    move_to_coord(pipette, pipette, position_sample, offset=(x_shaking,x_shaking,0), minimum_z_height=None)
    move_to_coord(pipette, position_sample, offset=(0,x_shaking,0), minimum_z_height=None)
    move_to_coord(pipette, position_sample, offset=(0,0,0), minimum_z_height=None)
    
move_to_coord(pipette, position_bsa, offset=(0,0,offset_z_reagents), minimum_z_height=minimum_z_height)


Cannot pick up tip with a tip attached
no pipette attached


Moving to: (17.48, 214.07, -8.43)


Moving to: (252, 133, 131)


Moving to: (17.48, 214.07, -8.43)


# Wait 10 minutes and remove BSA 

In [35]:
# remove PBS from sample
t_wait_wash=1
move_to_coord(pipette, position_sample, offset=(0,0,1), minimum_z_height=minimum_z_height)

pipette.aspirate(200)
move_to_coord(pipette, position_trash, offset=(0,0,0), minimum_z_height=minimum_z_height)
pipette.dispense()
pipette.blow_out()

# wait for 60s
time.sleep(t_wait_wash)

Moving to: (252, 133, 131)


Moving to: (36.78, 252.67000000000002, 11.57)


# Take a snapshot of the cells

In [38]:
def autofocus_coarse(dz=1000, nz=11, is_uselight=False):
    if is_uselight:
        microscope.set_laser_led(0,1)
        time.sleep(.2)
    payload = {'dz': list(np.linspace(-dz,dz,nz))}
    print("Focusing: "+str(payload))
    microscope.extensions["org.openflexure.autofocus"]["autofocus"].post_json(payload)
    if is_uselight:
        microscope.set_laser_led(0,0)
    return microscope.position['z']


#autofocus_coarse(3000,7,is_uselight=True)
takesnapshot(pipette_8, position_sample_light, offset=(0,0,0))

print(position_sample_light)
microscope.set_laser_led(0,0)


Please add a pipette first!
Moving to: (252, 166, 200)
(252, 166, 200)


# Grab and Add Primary Antibody

In [100]:
# pkcup primary
#i_pipette = pickup_fresh_pipette_tip(i_pipette, pipette, tiprack)
V_aspirate = 100

move_to_coord(pipette, position_primary, offset=(0,0,offset_z_reagents), minimum_z_height=minimum_z_height)
pipette.aspirate(V_aspirate)

# add primary to sample
move_to_coord(pipette, position_sample, offset=(0,0,1), minimum_z_height=minimum_z_height)
pipette.dispense()
pipette.mix(3)

# shake a bit
x_shaking = 1
for _ in range(0):
    move_to_coord(pipette, position_sample, offset=(x_shaking,0,0), minimum_z_height=None)
    move_to_coord(pipette, position_sample, offset=(x_shaking,x_shaking,0), minimum_z_height=None)
    move_to_coord(pipette, position_sample, offset=(0,x_shaking,0), minimum_z_height=None)
    move_to_coord(pipette, position_sample, offset=(0,0,0), minimum_z_height=None)
    
# take image 
takesnapshot(pipette_8, position_sample_light, offset=(0,0,0))



Moving to: (17.48, 252.67000000000002, -3.4299999999999997)


Moving to: (252, 133, 131)


TypeError: move_to_coord() missing 1 required positional argument: 'position_xyz'

Now we need to wait until the antibody bind to the cell, in the meantime we want to continously capture images 


In [ ]:
time_primary = time.time()
t_wait_primary = 120
t_period_imaging = 60

i_image = 0
t_last_image = 0
while time.time()-time_primary<t_wait_primary*60:
    
    if time.time()-t_last_image > t_period_imaging*60:
        # do time lapse imaging
        print("Taking image..."+str(i_image)
        # take image 
        move_to_coord(pipette, position_sample_light, offset=(2,-2,0), minimum_z_height=minimum_z_height)
        microscope.capture_image_to_disk()
        t_last_image = time.time()
        i_image += 1



# Wash sample 3x 

In [ ]:
pipette.home()
i_pipette = pickup_fresh_pipette_tip(i_pipette, pipette, tiprack)

# add primary to sample
V_aspirate = 100
t_wait_wash = 1 # 60

# remove liquid from sample 
move_to_coord(pipette, position_sample, offset=(0,0,0), minimum_z_height=minimum_z_height)
pipette.aspirate(V_aspirate)

# trash liquid
move_to_coord(pipette, position_trash, offset=(0,0,-15), minimum_z_height=minimum_z_height)
pipette.dispense(V_aspirate)

for i in range(3):
    # grab PBS
    move_to_coord(pipette, position_pbs, offset=(0,0,-15), minimum_z_height=minimum_z_height)
    pipette.aspirate(V_aspirate)

    # add PBS to sample
    move_to_coord(pipette, position_sample, offset=(0,0,1), minimum_z_height=minimum_z_height)
    pipette.dispense()
    pipette.mix(1)
    
    
    # remove PBS from sample
    pipette.aspirate(V_aspirate+50)
    move_to_coord(pipette, position_trash, offset=(0,0,0), minimum_z_height=minimum_z_height)
    pipette.dispense()
    pipette.blow_out()
    
    # wait for 60s
    time.sleep(t_wait_wash)

# Grab and Add Secondary Antibody

In [ ]:
# pkcup secondary
i_pipette = pickup_fresh_pipette_tip(i_pipette, pipette, tiprack)
V_aspirate = 100

move_to_coord(pipette, position_secondary, offset=(0,0,offset_z_reagents), minimum_z_height=minimum_z_height)
pipette.aspirate(V_aspirate)

# add primary to sample
move_to_coord(position_sample, offset=(0,0,0), minimum_z_height=minimum_z_height)
pipette.dispense()
pipette.mix(3)

# shake a bit
x_shaking = 1
for _ in range(0):
    move_to_coord(pipette, position_sample, offset=(x_shaking,0,0), minimum_z_height=None)
    move_to_coord(pipette, position_sample, offset=(x_shaking,x_shaking,0), minimum_z_height=None)
    move_to_coord(pipette, position_sample, offset=(0,x_shaking,0), minimum_z_height=None)
    move_to_coord(pipette, position_sample, offset=(0,0,0), minimum_z_height=None)
    
# take image 
move_to_coord(position_sample_light, offset=(2,-2,0), minimum_z_height=minimum_z_height)
microscope.capture_image_to_disk()

# move light somewhere else
move_to_coord((20,20,150), offset=(2,-2,0), minimum_z_height=minimum_z_height)


# Wash 3 times

In [ ]:
i_pipette = pickup_fresh_pipette_tip(i_pipette, pipette, tiprack)

# add primary to sample
V_aspirate = 100
t_wait_wash = 1 # 60

# remove liquid from sample 
move_to_coord(pipette, position_sample, offset=(0,0,0), minimum_z_height=minimum_z_height)
pipette.aspirate(V_aspirate)

# trash liquid
move_to_coord(pipette, position_trash, offset=(0,0,-15), minimum_z_height=minimum_z_height)
pipette.dispense(V_aspirate)

for i in range(3):
    # grab PBS
    move_to_coord(pipette, position_pbs, offset=(0,0,-15), minimum_z_height=minimum_z_height)
    pipette.aspirate(V_aspirate)

    # add PBS to sample
    move_to_coord(pipette, position_sample, offset=(0,0,1), minimum_z_height=minimum_z_height)
    pipette.dispense()
    pipette.mix(1)
    
    
    # remove PBS from sample
    pipette.aspirate(V_aspirate+50)
    move_to_coord(pipette, position_trash, offset=(0,0,0), minimum_z_height=minimum_z_height)
    pipette.dispense()
    pipette.blow_out()
    
    # wait for 60s
    time.sleep(t_wait_wash)

# Add MEA and perform imaging

In [ ]:
# pkcup secondary
i_pipette = pickup_fresh_pipette_tip(i_pipette, pipette, tiprack, -3)
V_aspirate = 100


move_to_coord(pipette, position_mea, offset=(0,0,offset_z_reagents), minimum_z_height=minimum_z_height)
pipette.aspirate(V_aspirate)

# add primary to sample
move_to_coord(pipette, position_sample, offset=(0,0,0), minimum_z_height=minimum_z_height)
pipette.dispense()
pipette.mix(3)

# shake a bit
x_shaking = 1
for _ in range(0):
    move_to_coord(pipette, position_sample, offset=(x_shaking,0,0), minimum_z_height=None)
    move_to_coord(pipette, position_sample, offset=(x_shaking,x_shaking,0), minimum_z_height=None)
    move_to_coord(pipette, position_sample, offset=(0,x_shaking,0), minimum_z_height=None)
    move_to_coord(pipette, position_sample, offset=(0,0,0), minimum_z_height=None)
    
# take image 
move_to_coord(pipette, position_sample_light, offset=(2,-2,0), minimum_z_height=minimum_z_height)
microscope.capture_image_to_disk()

# move light somewhere else
move_to_coord(pipette, (20,20,150), offset=(2,-2,0), minimum_z_height=minimum_z_height)


# Turn on the laser and start imaging


In [37]:
print(microscope.extensions['org.openflexure.video_extension'].__dict__)

def record_video(video_format= "h264", video_framerate=30,video_length=100):
    payload = {
      "move_x": 0,
      "video_format": video_format,
      "video_framerate": video_framerate,
      "video_length": video_length
    }
    microscope.extensions['org.openflexure.video_extension']['video_api'].post_json(payload)
microscope.set_laser_led(0,1)
record_video(video_length=1)

microscope.set_laser_led(512,0)
record_video(video_length=1)

microscope.set_laser_led(0,1)

{'extension_dict': {'pythonObject': '<labthings.extensions.BaseExtension object at 0x7f345e11d0>', 'description': 'Parent class for all extensions. Handles binding route views and forms. ', 'meta': {'gui': {'icon': 'videocam', 'forms': [{'name': 'Record a video', 'route': '/org.openflexure.video-extension/video', 'isTask': True, 'isCollapsible': False, 'submitLabel': 'Start recording', 'schema': [{'fieldType': 'numberInput', 'name': 'video_length', 'label': 'Length (seconds) of video', 'min': 0.1, 'step': 0.1, 'default': 100}, {'fieldType': 'numberInput', 'name': 'video_framerate', 'label': 'Framerate (Frames Per Second)', 'min': 1, 'max': 90, 'step': 1, 'default': 30}, {'fieldType': 'selectList', 'name': 'video_format', 'label': 'Video format', 'value': 'h264', 'options': ['h264', 'mjpeg', 'yuv', 'rgb', 'rgba', 'bgr', 'bgra']}, {'fieldType': 'numberInput', 'name': 'move_x', 'label': 'Steps to move in X', 'min': 0, 'step': 100, 'default': 0}]}], 'id': 'org.openflexure.video_extension',

In [38]:

microscope.set_laser_led(1024,1)

# Joging the robot


In [39]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [40]:
def f(x):
    return x

In [41]:
interact(f, x=10);

10

In [ ]:
N_steps_x=5
np.array()